In [401]:
import pynq
import time
import numpy as np
import socket

In [402]:
HOST = "192.168.0.160"  # The server's hostname or IP address
PORT = 65434  # The port used by the server

In [403]:
np.set_printoptions(threshold=np.inf)

In [404]:
bitstream = "trial2_removedwarning_des.bit"
overlay = pynq.Overlay(bitstream)
dut = overlay.tiled_conv_0

In [405]:
#--------------------------------------------------------------------------
# Input, Layers and Parameters, Output
#--------------------------------------------------------------------------

input_feature_map = np.zeros((1,28,28), dtype=np.int16)
output_feature_map = np.zeros((10), dtype=np.int16)
#conv_layer_weights = np.zeros((1, 1, 5, 5), dtype=np.int16)
mse_flp = np.zeros((1), dtype=np.int16)

In [406]:
target_output_set_flp = np.array([[0,0,0,0,0,0,0,0,0,1],[0,0,1,0,0,0,0,0,0,0]], dtype=np.float32)
#input_feature_set_linear_flp = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,13,7,0,0,0,0,0,0,2,93,244,207,8,0,0,0,0,0,0,0,0,0,0,0,0,0,15,157,254,197,0,0,0,0,0,18,131,254,254,248,11,0,0,0,0,0,0,0,0,0,0,0,0,9,220,254,254,230,104,0,0,0,65,216,254,254,254,218,8,0,0,0,0,0,0,0,0,0,0,0,0,169,254,254,231,126,40,11,70,180,254,254,254,254,254,90,0,0,0,0,0,0,0,0,0,0,0,40,183,251,254,226,81,70,180,229,254,254,254,254,254,254,115,3,0,0,0,0,0,0,0,0,0,0,7,208,254,254,255,254,254,254,255,254,254,254,254,254,254,190,3,0,0,0,0,0,0,0,0,0,0,0,53,254,254,254,254,254,253,250,212,169,125,167,254,254,241,35,0,0,0,0,0,0,0,0,0,0,0,0,150,254,254,181,77,77,48,0,0,0,128,254,254,253,57,0,0,0,0,0,0,0,0,0,0,0,0,0,14,157,195,29,0,0,0,0,0,75,248,254,254,139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,217,254,254,170,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,217,254,254,214,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,113,254,254,238,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,89,254,254,239,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,141,254,254,240,106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66,252,254,252,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,250,254,252,171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,249,254,254,210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56,223,254,254,189,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,231,254,254,211,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,226,254,254,209,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,146,254,255,251,95,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,97,234,254,254,232,254,254,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89,140,254,254,174,67,33,200,254,190,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,108,253,254,235,51,1,0,0,12,254,253,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,216,254,244,55,0,0,0,0,6,213,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,254,254,132,0,0,0,0,0,0,168,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,254,243,34,0,0,0,0,0,0,168,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,254,157,0,0,0,0,0,0,0,168,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,228,254,105,0,0,0,0,0,0,7,228,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,254,254,87,0,0,0,0,0,0,10,254,246,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,254,254,9,0,0,0,0,0,0,10,254,210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,254,254,9,0,0,0,0,0,0,105,254,91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,219,254,9,0,0,0,0,0,24,230,254,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,216,254,9,0,0,0,0,0,84,254,251,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,216,254,36,0,0,0,0,22,208,251,94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,129,254,120,0,0,0,3,140,254,229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,254,222,17,0,0,91,254,236,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,235,254,134,21,119,237,254,124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,249,254,234,252,254,172,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,237,254,254,133,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], dtype=np.float32)
input_feature_set_linear_flp = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,148,210,253,253,113,87,148,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,232,252,253,189,210,252,252,253,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,57,242,252,190,65,5,12,182,252,253,116,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,252,252,183,14,0,0,92,252,252,225,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,132,253,252,146,14,0,0,0,215,252,252,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,126,253,247,176,9,0,0,8,78,245,253,129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,232,252,176,0,0,0,36,201,252,252,169,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,252,252,30,22,119,197,241,253,252,251,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,231,252,253,252,252,252,226,227,252,231,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,235,253,217,138,42,24,192,252,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,62,255,253,109,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,253,252,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,253,252,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,253,252,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106,253,252,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,255,253,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,218,252,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,252,189,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,184,252,170,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,25,100,122,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,151,208,252,252,252,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,152,244,252,253,224,211,252,232,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,152,239,252,252,252,216,31,37,252,252,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,252,252,252,252,217,29,0,37,252,252,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181,252,252,220,167,30,0,0,77,252,252,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,128,58,22,0,0,0,0,100,252,252,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,157,252,252,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110,121,122,121,202,252,194,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,53,179,253,253,255,253,253,228,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,54,227,252,243,228,170,242,252,252,231,117,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,78,252,252,125,59,0,18,208,252,252,252,252,87,7,0,0,0,0,0,0,0,0,0,0,0,0,5,135,252,252,180,16,0,21,203,253,247,129,173,252,252,184,66,49,49,0,0,0,0,0,0,0,0,3,136,252,241,106,17,0,53,200,252,216,65,0,14,72,163,241,252,252,223,0,0,0,0,0,0,0,0,105,252,242,88,18,73,170,244,252,126,29,0,0,0,0,0,89,180,180,37,0,0,0,0,0,0,0,0,231,252,245,205,216,252,252,252,124,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,207,252,252,252,252,178,116,36,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,93,143,121,23,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
#input_feature_set_linear_flp = np.array([x/255 for x in input_feature_set_linear_flp], dtype=np.float32)
conv_layer_weights_flp = np.array([[-0.0801, -0.1498,  0.1818,  0.2058, -0.0423],[-0.1670,  0.3049,  0.5410,  0.2155,  0.3863],[0.1328,  0.5504,  0.8918,  0.6554,  0.3264],[0.4062,  0.8718,  1.1696,  1.0350,  0.3345],[0.2787,  0.5149,  0.5678,  0.7517,  0.2824]], dtype=np.float32)
print(conv_layer_weights_flp.shape)
print(input_feature_set_linear_flp)
print(target_output_set_flp.shape)

(5, 5)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0  55 148 210 253 253 113  87 148
   55   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0  87 232 252 253 189 210 2

In [407]:
# Author: Rishov Sarkar

# Convert floating-point array to fixed-point
# Sample Usage:
# to_fixed_point(A_fixp, A_float, iwidth=3)
def to_fixed_point(dst, src, *, width=None, iwidth, signed=True):
    if width is None:
        width = dst.dtype.itemsize * 8

    fwidth = width - iwidth
    epsilon = 1.0 / (2.0 ** fwidth)
    min_ = -1.0 * (2.0 ** (iwidth - 1)) if signed else 0.0
    max_ = (2.0 ** (iwidth - (1 if signed else 0))) - epsilon

    src = np.copy(src)
    src = src.reshape(dst.shape)
    src[src < min_] = min_
    src[src > max_] = max_
    if signed:
        src[src < 0] += (2 ** iwidth)
    dst[:] = np.around(src * (2.0 ** fwidth)).astype(dst.dtype)
    
# Convert fixed-point array back to floating point
# Sample Usage:
# B_float = from_fixed_point(B_fixp, iwidth=3)
def from_fixed_point(src, *, width=None, iwidth, signed=True):
    if width is None:
        width = src.dtype.itemsize * 8

    fwidth = width - iwidth
    src = np.array(src, dtype=np.int64)
    if signed:
        src[src >= (2 ** (width - 1))] -= (2 ** width)
    return src / (2.0 ** fwidth)

In [408]:
#Converting to fixed point
conv_layer_weights = np.zeros((5,5), dtype=np.int16)
target_output_set = np.zeros((2,10), dtype=np.int16)
input_feature_set_linear = np.zeros((2,784), dtype=np.int16)
mse = np.zeros((1), dtype=np.int16)

to_fixed_point(conv_layer_weights, conv_layer_weights_flp, iwidth=3)
to_fixed_point(target_output_set, target_output_set_flp, iwidth=3)
to_fixed_point(input_feature_set_linear, input_feature_set_linear_flp, iwidth=3)
to_fixed_point(mse, mse_flp, iwidth=3)

In [409]:
print("Beginning Inference...")

Beginning Inference...


In [410]:
def establish_connection(s):
    try:
        s.connect((HOST, PORT))
        print("Connection established!")
        
    except ConnectionRefusedError:
        print("Connection refused. Retrying in 5 seconds...")
        time.sleep(5)
        establish_connection(s)

In [411]:
arr=[]
fc_params_flp=[]

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    establish_connection(s)
    
    for i in range(144*10):
        result_bytes = s.recv(8)
        result = np.frombuffer(result_bytes)#, dtype=np.float64)
        arr= result.copy()
        fc_params_flp.append(arr)
    

fc_params_flp = [x for y in fc_params_flp for x in y]
print(type(fc_params_flp))
#fc_params = np.array(fc_params, dtype= np.int16)
print("Array received from the server: ", fc_params_flp)

Connection established!
<class 'list'>
Array received from the server:  [-0.079887666, 0.037666745, -0.0646291, 0.047336746, 0.03578022, 0.06894379, -0.0472288, 0.05440499, 0.038119763, -0.032299273, -0.022049926, -0.03805529, -0.050355196, 0.010031178, -0.020587714, -0.07619561, -0.046548553, -0.03449374, -0.0808194, -0.06223328, -0.034093734, -0.073432885, 0.11752969, 0.03245147, -0.028118867, -0.022356045, -0.03058101, -0.049455393, -0.07542431, -0.0500957, 0.014127678, 0.05404829, -0.011861383, 0.016889187, 0.027145976, -0.029775344, -0.033707034, 0.032428864, 0.03946386, 0.018680539, 0.057818286, 0.03424405, 0.006733727, -0.018292457, -0.0504974, 0.017603137, -0.016543087, -0.06486203, -0.014263901, 0.00832637, 0.0555361, -0.053385425, 0.052097365, -0.068347774, -0.07183572, -0.034721006, 0.074058704, -0.09424744, -0.059633873, -0.16036846, 0.023080794, 0.08541336, 0.03828963, -0.084177956, -0.083432324, -0.054616723, -0.06185484, -0.028995313, -0.022058094, -0.15541133, 0.0639438

In [412]:
fc_params = np.zeros((1440), dtype= np.int16)
to_fixed_point(fc_params, fc_params_flp, iwidth=3)
fc_params= fc_params.reshape(144,10)

print((fc_params))

[[ -654   309  -529   388   293   565  -387   446   312  -265]
 [ -181  -312  -413    82  -169  -624  -381  -283  -662  -510]
 [ -279  -602   963   266  -230  -183  -251  -405  -618  -410]
 [  116   443   -97   138   222  -244  -276   266   323   153]
 [  474   281    55  -150  -414   144  -136  -531  -117    68]
 [  455  -437   427  -560  -588  -284   607  -772  -489 -1314]
 [  189   700   314  -690  -683  -447  -507  -238  -181 -1273]
 [  524   140  -527   319  -667  -513   230  -204    79 -1609]
 [ -490   115   593  -286  -452 -1233  -113 -1193  -397 -1364]
 [ -168   688   862   233   433   368  -214  -871   493  -807]
 [  616   908   106    52   195   241   323   125   395  -840]
 [   -7   804   395   435  -139   385   -78   236   575   250]
 [  518   -20    32   196   519   134    60   434  -634  -506]
 [ -375  -805   698   196   -14  -238   666  -779  -356  -440]
 [ -491  -648   229   873   -91  -187   234   -57  -619  -872]
 [  385  -459  -358   607    32  -283   620 -1019  -863

In [413]:
fc_params_bkp = np.zeros((1440), dtype= np.int16)
to_fixed_point(fc_params_bkp, fc_params_flp, iwidth=3)
fc_params_bkp= fc_params_bkp.reshape(144,10)

In [414]:
#Call kernel function here
for i in range(2):
    for loop1 in range(1):
        for loop2 in range(28):
            for loop3 in range(28):
                input_feature_map[loop1][loop2][loop3] = input_feature_set_linear[i][loop1 * 28 * 28 + loop2 * 28 + loop3]
        
    print("After "+str(i+1)+" image : ")
    
    #print((fc_params))
    
    #Allocating buffers
    input_feature_map_buf = pynq.allocate((1, 28, 28), dtype='int16')
    conv_layer_weights_buf = pynq.allocate((1, 1, 5, 5), dtype='int16')
    fc_params_buf = pynq.allocate((144,10), dtype='int16')
    target_output_set_buf = pynq.allocate((2, 10), dtype='int16')
    mse_buf = pynq.allocate((1), dtype='int16')
    output_feature_map_buf = pynq.allocate((10), dtype='int16')

    input_feature_map_buf[:] = input_feature_map
    conv_layer_weights_buf[:] = conv_layer_weights
    fc_params_buf[:] = fc_params
    target_output_set_buf[:] = target_output_set[i]
    mse_buf[:] = mse
    
    input_feature_map_buf.sync_to_device()
    conv_layer_weights_buf.sync_to_device()
    fc_params_buf.sync_to_device()
    target_output_set_buf.sync_to_device()
    mse_buf.sync_to_device()
    
    dut.register_map
    
    dut.register_map.input_feature_map_1 = input_feature_map_buf.device_address & 0xFFFFFFFF
    dut.register_map.input_feature_map_2 = input_feature_map_buf.device_address >> 32

    dut.register_map.layer_weights_1 = conv_layer_weights_buf.device_address & 0xFFFFFFFF
    dut.register_map.layer_weights_2 = conv_layer_weights_buf.device_address >> 32

    dut.register_map.linear_weights_1 = fc_params_buf.device_address & 0xFFFFFFFF
    dut.register_map.linear_weights_2 = fc_params_buf.device_address >> 32
    
    dut.register_map.target_output_1 = target_output_set_buf.device_address & 0xFFFFFFFF
    dut.register_map.target_output_2 = target_output_set_buf.device_address >> 32
    
    dut.register_map.mse_1 = mse_buf.device_address & 0xFFFFFFFF
    dut.register_map.mse_2 = mse_buf.device_address >> 32
    
    dut.register_map.output_feature_map_1 = output_feature_map_buf.device_address & 0xFFFFFFFF
    dut.register_map.output_feature_map_2 = output_feature_map_buf.device_address >> 32
    
    start_time = time.time()

    dut.register_map.CTRL.AP_START = 1
    dut.register_map.CTRL[4] = 1
    while not dut.register_map.CTRL.AP_DONE: pass
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Kernel completed in {duration * 1000:.2f}ms')
    fc_params_buf.sync_from_device()
    
    #fc_param= np.zeros((144,10))
   # for a in range(144):
   #     for b in range(10):
    #print(fc_params-fc_params_buf)
    fc_params[:]= fc_params_buf
    output_feature_map_buf.sync_from_device()
    
    print(from_fixed_point(output_feature_map_buf, iwidth=3))
    


After 1 image : 
Kernel completed in 0.49ms
[0.01953125 0.01953125 0.01953125 0.01953125 0.23242188 0.01953125
 0.01953125 0.35351562 0.01953125 0.26757812]
After 2 image : 
Kernel completed in 0.50ms
[0.015625   0.74023438 0.015625   0.015625   0.015625   0.12109375
 0.015625   0.015625   0.015625   0.015625  ]


In [415]:
print(fc_params-fc_params_bkp)


[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0

In [416]:
#Sending the data back
rx_fc_param= from_fixed_point(fc_params_buf, iwidth=3)
rx_fc_param_ser= rx_fc_param.flatten()

rx_fc_param_bytes = rx_fc_param.tobytes()

#c_1_bytes = np.array(c_ser, dtype=np.int16).tobytes()
#c_2_bytes = np.array(c_ser[10000:20000], dtype=np.int16).tobytes()

print("Array Sent: ", rx_fc_param)

PORT = 6002

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    establish_connection(s)
    data= rx_fc_param_bytes
    s.sendall(data)


Array Sent:  [[-7.98339844e-02  3.77197266e-02 -6.45751953e-02  4.73632812e-02
   3.57666016e-02  6.89697266e-02 -4.72412109e-02  5.44433594e-02
   3.80859375e-02 -3.23486328e-02]
 [-2.20947266e-02 -3.80859375e-02 -5.04150391e-02  1.00097656e-02
  -2.06298828e-02 -7.61718750e-02 -4.65087891e-02 -3.45458984e-02
  -8.08105469e-02 -6.22558594e-02]
 [-3.40576172e-02 -7.34863281e-02  1.17553711e-01  3.24707031e-02
  -2.80761719e-02 -2.23388672e-02 -3.06396484e-02 -4.94384766e-02
  -7.54394531e-02 -5.00488281e-02]
 [ 1.41601562e-02  5.40771484e-02 -1.18408203e-02  1.68457031e-02
   2.70996094e-02 -2.97851562e-02 -3.36914062e-02  3.24707031e-02
   3.94287109e-02  1.86767578e-02]
 [ 5.78613281e-02  3.43017578e-02  6.71386719e-03 -1.83105469e-02
  -5.05371094e-02  1.75781250e-02 -1.66015625e-02 -6.48193359e-02
  -1.42822266e-02  8.30078125e-03]
 [ 5.55419922e-02 -5.33447266e-02  5.21240234e-02 -6.83593750e-02
  -7.17773438e-02 -3.46679688e-02  7.40966797e-02 -9.42382812e-02
  -5.96923828e-02 -1

In [417]:
PORT= 6006

In [418]:
arr=[]
fc_params_flp=[]

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    establish_connection(s)
    
    for i in range(144*10):
        result_bytes = s.recv(8)
        result = np.frombuffer(result_bytes)#, dtype=np.float64)
        arr= result.copy()
        fc_params_flp.append(arr)
    

fc_params_flp = [x for y in fc_params_flp for x in y]
print(type(fc_params_flp))
#fc_params = np.array(fc_params, dtype= np.int16)
print("Array received from the server: ", fc_params_flp)

Connection refused. Retrying in 5 seconds...
Connection refused. Retrying in 5 seconds...
Connection established!
<class 'list'>
Array received from the server:  [-0.079833984375, 0.0377197265625, -0.0645751953125, 0.04736328125, 0.0357666015625, 0.0689697265625, -0.0472412109375, 0.054443359375, 0.0380859375, -0.0323486328125, -0.0220947265625, -0.0380859375, -0.0504150390625, 0.010009765625, -0.0206298828125, -0.076171875, -0.0465087890625, -0.0345458984375, -0.080810546875, -0.062255859375, -0.0340576171875, -0.073486328125, 0.1175537109375, 0.032470703125, -0.028076171875, -0.0223388671875, -0.0306396484375, -0.0494384765625, -0.075439453125, -0.050048828125, 0.01416015625, 0.0540771484375, -0.0118408203125, 0.016845703125, 0.027099609375, -0.02978515625, -0.03369140625, 0.032470703125, 0.0394287109375, 0.0186767578125, 0.057861328125, 0.0343017578125, 0.0067138671875, -0.018310546875, -0.050537109375, 0.017578125, -0.0166015625, -0.0648193359375, -0.0142822265625, 0.00830078125, 0

In [419]:
fc_params = np.zeros((1440), dtype= np.int16)
to_fixed_point(fc_params, fc_params_flp, iwidth=3)
fc_params= fc_params.reshape(144,10)

print((fc_params))

[[ -654   309  -529   388   293   565  -387   446   312  -265]
 [ -181  -312  -413    82  -169  -624  -381  -283  -662  -510]
 [ -279  -602   963   266  -230  -183  -251  -405  -618  -410]
 [  116   443   -97   138   222  -244  -276   266   323   153]
 [  474   281    55  -150  -414   144  -136  -531  -117    68]
 [  455  -437   427  -560  -588  -284   607  -772  -489 -1314]
 [  189   700   314  -690  -683  -447  -507  -238  -181 -1273]
 [  524   140  -527   319  -667  -513   230  -204    79 -1609]
 [ -490   115   593  -286  -452 -1233  -113 -1193  -397 -1364]
 [ -168   688   862   233   433   368  -214  -871   493  -807]
 [  616   908   106    52   195   241   323   125   395  -840]
 [   -7   804   395   435  -139   385   -78   236   575   250]
 [  518   -20    32   196   519   134    60   434  -634  -506]
 [ -375  -805   698   196   -14  -238   666  -779  -356  -440]
 [ -491  -648   229   873   -91  -187   234   -57  -619  -872]
 [  385  -459  -358   607    32  -283   620 -1019  -863

In [420]:
fc_params_bkp = np.zeros((1440), dtype= np.int16)
to_fixed_point(fc_params_bkp, fc_params_flp, iwidth=3)
fc_params_bkp= fc_params_bkp.reshape(144,10)

In [421]:
#Call kernel function here
for i in range(2):
    for loop1 in range(1):
        for loop2 in range(28):
            for loop3 in range(28):
                input_feature_map[loop1][loop2][loop3] = input_feature_set_linear[i][loop1 * 28 * 28 + loop2 * 28 + loop3]
        
    print("After "+str(i+1)+" image : ")
    
    #print((fc_params))
    
    #Allocating buffers
    input_feature_map_buf = pynq.allocate((1, 28, 28), dtype='int16')
    conv_layer_weights_buf = pynq.allocate((1, 1, 5, 5), dtype='int16')
    fc_params_buf = pynq.allocate((144,10), dtype='int16')
    target_output_set_buf = pynq.allocate((2, 10), dtype='int16')
    mse_buf = pynq.allocate((1), dtype='int16')
    output_feature_map_buf = pynq.allocate((10), dtype='int16')

    input_feature_map_buf[:] = input_feature_map
    conv_layer_weights_buf[:] = conv_layer_weights
    fc_params_buf[:] = fc_params
    target_output_set_buf[:] = target_output_set[i]
    mse_buf[:] = mse
    
    input_feature_map_buf.sync_to_device()
    conv_layer_weights_buf.sync_to_device()
    fc_params_buf.sync_to_device()
    target_output_set_buf.sync_to_device()
    mse_buf.sync_to_device()
    
    dut.register_map
    
    dut.register_map.input_feature_map_1 = input_feature_map_buf.device_address & 0xFFFFFFFF
    dut.register_map.input_feature_map_2 = input_feature_map_buf.device_address >> 32

    dut.register_map.layer_weights_1 = conv_layer_weights_buf.device_address & 0xFFFFFFFF
    dut.register_map.layer_weights_2 = conv_layer_weights_buf.device_address >> 32

    dut.register_map.linear_weights_1 = fc_params_buf.device_address & 0xFFFFFFFF
    dut.register_map.linear_weights_2 = fc_params_buf.device_address >> 32
    
    dut.register_map.target_output_1 = target_output_set_buf.device_address & 0xFFFFFFFF
    dut.register_map.target_output_2 = target_output_set_buf.device_address >> 32
    
    dut.register_map.mse_1 = mse_buf.device_address & 0xFFFFFFFF
    dut.register_map.mse_2 = mse_buf.device_address >> 32
    
    dut.register_map.output_feature_map_1 = output_feature_map_buf.device_address & 0xFFFFFFFF
    dut.register_map.output_feature_map_2 = output_feature_map_buf.device_address >> 32
    
    start_time = time.time()

    dut.register_map.CTRL.AP_START = 1
    dut.register_map.CTRL[4] = 1
    while not dut.register_map.CTRL.AP_DONE: pass
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Kernel completed in {duration * 1000:.2f}ms')
    fc_params_buf.sync_from_device()
    
    #fc_param= np.zeros((144,10))
   # for a in range(144):
   #     for b in range(10):
    #print(fc_params-fc_params_buf)
    fc_params[:]= fc_params_buf
    output_feature_map_buf.sync_from_device()
    
    print(from_fixed_point(output_feature_map_buf, iwidth=3))

After 1 image : 
Kernel completed in 0.48ms
[0.02148438 0.02148438 0.02148438 0.02148438 0.265625   0.02148438
 0.02148438 0.24414062 0.02148438 0.3359375 ]
After 2 image : 
Kernel completed in 0.49ms
[0.02148438 0.66210938 0.02148438 0.02148438 0.02148438 0.15625
 0.02148438 0.02148438 0.02148438 0.02148438]


In [422]:
print(fc_params-fc_params_bkp)

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0

In [423]:
overlay.free()

In [424]:
#Sending the data back
rx_fc_param= from_fixed_point(fc_params_buf, iwidth=3)
rx_fc_param_ser= rx_fc_param.flatten()

rx_fc_param_bytes = rx_fc_param.tobytes()

print("Array Sent: ", rx_fc_param)

PORT = 6010

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    establish_connection(s)
    data= rx_fc_param_bytes
    s.sendall(data)

Array Sent:  [[-7.98339844e-02  3.77197266e-02 -6.45751953e-02  4.73632812e-02
   3.57666016e-02  6.89697266e-02 -4.72412109e-02  5.44433594e-02
   3.80859375e-02 -3.23486328e-02]
 [-2.20947266e-02 -3.80859375e-02 -5.04150391e-02  1.00097656e-02
  -2.06298828e-02 -7.61718750e-02 -4.65087891e-02 -3.45458984e-02
  -8.08105469e-02 -6.22558594e-02]
 [-3.40576172e-02 -7.34863281e-02  1.17553711e-01  3.24707031e-02
  -2.80761719e-02 -2.23388672e-02 -3.06396484e-02 -4.94384766e-02
  -7.54394531e-02 -5.00488281e-02]
 [ 1.41601562e-02  5.40771484e-02 -1.18408203e-02  1.68457031e-02
   2.70996094e-02 -2.97851562e-02 -3.36914062e-02  3.24707031e-02
   3.94287109e-02  1.86767578e-02]
 [ 5.78613281e-02  3.43017578e-02  6.71386719e-03 -1.83105469e-02
  -5.05371094e-02  1.75781250e-02 -1.66015625e-02 -6.48193359e-02
  -1.42822266e-02  8.30078125e-03]
 [ 5.55419922e-02 -5.33447266e-02  5.21240234e-02 -6.83593750e-02
  -7.17773438e-02 -3.46679688e-02  7.40966797e-02 -9.42382812e-02
  -5.96923828e-02 -1